<a href="https://colab.research.google.com/github/anilkulkarni87/stock-markets-analytics-zoomcamp/blob/main/Module2_Colab_Working_with_the_data_HW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install yfinance

In [5]:
# IMPORTS
import numpy as np
import pandas as pd
import requests

#Fin Data Sources
import yfinance as yf
import pandas_datareader as pdr

#Data viz
import plotly.graph_objs as go
import plotly.express as px

import time
from datetime import date

# for graphs
import matplotlib.pyplot as plt


# Question 1: IPO Filings Web Scraping and Data Processing

What's the total sum ($m) of 2023 filings that happened on Fridays?

Re-use the [Code Snippet 1] example to get the data from web for this endpoint: https://stockanalysis.com/ipos/filings/ Convert the 'Filing Date' to datetime(), 'Shares Offered' to float64 (if '-' is encountered, populate with NaNs). Define a new field 'Avg_price' based on the "Price Range", which equals to NaN if no price is specified, to the price (if only one number is provided), or to the average of 2 prices (if a range is given). You may be inspired by the function extract_numbers() in [Code Snippet 4], or you can write your own function to "parse" a string. Define a column "Shares_offered_value", which equals to "Shares Offered" * "Avg_price" (when both columns are defined; otherwise, it's NaN)

Find the total sum in $m (millions of USD, closest INTEGER number) for all filings during 2023, which happened on Fridays (Date.dt.dayofweek()==4). You should see 32 records in total, 25 of it is not null.

(additional: you can read about S-1 IPO filing to understand the context

In [27]:
# Calculate Avg_price and Shares_offered_value
def calculate_avg_price(price_range):
    if price_range == "-":
        return None
    prices = [float(price.strip('$')) for price in price_range.split(' - ')]
    return sum(prices) / len(prices)

In [29]:
import pandas as pd
import requests

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

# Get all records
url = "https://stockanalysis.com/ipos/filings/"
# url = "https://stockanalysis.com/ipos/2023/"
response = requests.get(url, headers=headers)

ipo_dfs = pd.read_html(response.text)
final_ipo=ipo_dfs[0]
final_ipo.head(1)
# convert to datetime
final_ipo['Filing Date'] = pd.to_datetime(final_ipo['Filing Date'], format='%b %d, %Y')
final_ipo["Shares Offered"] = pd.to_numeric(final_ipo["Shares Offered"].str.replace(",", ""), errors="coerce")

final_ipo["Avg_price"] = final_ipo["Price Range"].apply(calculate_avg_price)
final_ipo["Shares_offered_value"] = final_ipo["Shares Offered"] * final_ipo["Avg_price"]
final_ipo.head(20)

,Filing Date,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
0,2024-05-03,TBN,Tamboran Resources Corporation,-,NaN,NaN,NaN
1,2024-04-29,HWEC,"HW Electro Co., Ltd.",$3.00,3750000.0,3.00,11250000.0
2,2024-04-29,DTSQ,DT Cloud Star Acquisition Corporation,$10.00,6000000.0,10.00,60000000.0
3,2024-04-26,EURK,Eureka Acquisition Corp,$10.00,5000000.0,10.00,50000000.0
4,2024-04-26,HDL,Super Hi International Holding Ltd.,-,NaN,NaN,NaN
5,2024-04-22,DRJT,Derun Group Inc,$5.00,NaN,5.00,NaN
6,2024-04-19,GPAT,GP-Act III Acquisition Corp.,$10.00,25000000.0,10.00,250000000.0
7,2024-04-16,JLJT,Jialiang Holdings Ltd,$5.00,NaN,5.00,NaN
8,2024-04-15,GAUZ,Gauzy Ltd.,-,NaN,NaN,NaN
9,2024-04-12,BOW,Bowhead Specialty Holdings Inc.,-,NaN,NaN,NaN


In [36]:
# Filter for filings during 2023 that happened on Fridays
filtered_df = final_ipo[(final_ipo["Filing Date"].dt.year == 2023) & (final_ipo["Filing Date"].dt.dayofweek == 4)]
filtered_df
# Calculate the total sum in $m for all filtered filings
total_sum_millions_usd = int(filtered_df["Shares_offered_value"].sum())

print("Total sum in $m for all filings during 2023 that happened on Fridays:", total_sum_millions_usd)

Total sum in $m for all filings during 2023 that happened on Fridays: 285700000


# Question 2: IPOs "Fixed days hold" strategy
Find the optimal number of days X (between 1 and 30), where 75% quantile growth is the highest?

Reuse [Code Snippet 1] to retrieve the list of IPOs from 2023 and 2024 (from URLs: https://stockanalysis.com/ipos/2023/ and https://stockanalysis.com/ipos/2024/). Get all OHLCV daily prices for all stocks with an "IPO date" before March 1, 2024 ("< 2024-03-01") - 184 tickers (without 'RYZB'). Please remove 'RYZB', as it is no longer available on Yahoo Finance.

Sometimes you may need to adjust the symbol name (e.g., 'IBAC' on stockanalysis.com -> 'IBACU' on Yahoo Finance) to locate OHLCV prices for all stocks. Also, you can see the ticker changes using this link. Some of the tickers (like 'DYCQ' and 'LEGT') were on the market less than 30 days (11 and 21 days, respectively). Let's leave them in the dataset; it just means that you couldn't hold them for more days than they were listed.

Let's assume you managed to buy a new stock (listed on IPO) on the first day at the [Adj Close] price]. Your strategy is to hold for exactly X full days (where X is between 1 and 30) and sell at the "Adj. Close" price in X days (e.g., if X=1, you sell on the next day). Find X, when the 75% quantile growth (among 185 investments) is the highest.

HINTs:

You can generate 30 additional columns: growth_future_1d ... growth_future_30d, join that with the table of min_dates (first day when each stock has data on Yahoo Finance), and perform vector operations on the resulting dataset.
You can use the DataFrame.describe() function to get mean, min, max, 25-50-75% quantiles.
Additional:

You can also ensure that the mean and 50th percentile (median) investment returns are negative for most X values, implying a wager for a "lucky" investor who might be in the top 25%.
What's your recommendation: Do you suggest pursuing this strategy for an optimal X?


In [6]:
# Step 1: Retrieve the list of IPOs from 2023 and 2024
urls = ["https://stockanalysis.com/ipos/2023/", "https://stockanalysis.com/ipos/2024/"]
ipo_dfs = [pd.read_html(requests.get(url).text)[0] for url in urls]
ipo_df = pd.concat(ipo_dfs)
ipo_df['IPO Date'] = pd.to_datetime(ipo_df['IPO Date'], format='%b %d, %Y')
ipo_df = ipo_df[(ipo_df['IPO Date'] < '2024-03-01') & (ipo_df["Symbol"] != "RYZB")]

ipo_df.info()
ipo_df.head(5)

<class 'pandas.core.frame.DataFrame'>
Index: 184 entries, 0 to 63
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   IPO Date      184 non-null    datetime64[ns]
 1   Symbol        184 non-null    object        
 2   Company Name  184 non-null    object        
 3   IPO Price     184 non-null    object        
 4   Current       184 non-null    object        
 5   Return        184 non-null    object        
dtypes: datetime64[ns](1), object(5)
memory usage: 10.1+ KB


,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,$10.00,$10.05,0.50%
1,2023-12-19,LGCB,Linkage Global Inc,$4.00,$2.91,-27.25%
2,2023-12-15,ZKH,ZKH Group Limited,$15.50,$12.95,-16.45%
3,2023-12-15,BAYA,Bayview Acquisition Corp,$10.00,$10.18,1.80%
4,2023-12-14,INHD,Inno Holdings Inc.,$4.00,$0.62,-84.45%


In [8]:
# Step 2: Get all OHLCV daily prices for all stocks with an IPO date before March 1, 2024

tickers = ipo_df["Symbol"].tolist()
len(tickers)


184

In [12]:
#Ticker changes
# URL to fetch ticker changes mapping
url = "https://stockanalysis.com/actions/changes/"

# Fetch the HTML content
response = requests.get(url)

# Read HTML table into a DataFrame
ticker_changes_df = pd.read_html(response.text)[0]

# Convert DataFrame to dictionary
ticker_changes = dict(zip(ticker_changes_df["Old"].str.strip(), ticker_changes_df["New"].str.strip()))

print("Ticker changes mapping:")
print(ticker_changes)

Ticker changes mapping:
{'CONX': 'CNXX', 'FUV': 'FUVV', 'XLYO': 'XYLO', 'DCFCW': 'DCFWQ', 'MDGS': 'XYLO', 'GGE': 'GGEI', 'CBD': 'CBDBY', 'OSA': 'OSAP', 'ARRW': 'AILE', 'CNXA': 'YYAI', 'STRC': 'PDYN', 'KYCH': 'ZOOZ', 'VIEWW': 'VIWWQ', 'ASCA': 'NIVF', 'CASA': 'CASSQ', 'ONTX': 'TRAW', 'APAC': 'FAAS', 'WETG': 'NXTT', 'GEV.W': 'GEV', 'WNNR': 'ZPTA', 'CVCY': 'CWBC', 'TWOA': 'LPA', 'AGE': 'SER', 'CNDB': 'GCTS', 'FIXX': 'QTTB', 'DWAC': 'DJT', 'FSR': 'FSRN', 'FLT': 'CPAY', 'VJET': 'VJTTY', 'GRPH': 'LENZ', 'ADOC': 'COOT', 'HCMA': 'MRNO', 'SVFD': 'NITO', 'PUYI': 'HPH', 'ENCP': 'GTI', 'CYT': 'CYTT', 'DHCA': 'BNAI', 'XPDB': 'AIRJ', 'ESAC': 'ZEO', 'INPX': 'XTIA', 'SASI': 'NTRP', 'LBBB': 'NMHI', 'GIA': 'QTI', 'CPSI': 'TBRG', 'PEAK': 'DOC', 'BODY': 'BODI', 'BFX': 'BFXXQ', 'ARIZ': 'FUFU', 'POL': 'POLCQ', 'SZZL': 'CRML', 'ROI': 'ROII', 'TMST': 'MTUS', 'AMEH': 'ASTH', 'LCAA': 'LOT', 'JT': 'JTCHY', 'CHEA': 'SELX', 'LGST': 'TVGN', 'FLME': 'SOC', 'PBAX': 'CERO', 'SEAS': 'PRKS', 'QFTA': 'ATCH', 'AEY': 'AEYGQ

In [13]:
tickers = [ticker_changes.get(ticker, ticker) for ticker in tickers]
len(tickers)

184

In [39]:
# Fetch historical data for each ticker
from datetime import datetime, timedelta
start_date = (datetime.now() - timedelta(days=365)).strftime('%Y-%m-%d')
end_date = "2024-03-01"
stocks_df = pd.DataFrame({'A' : []})
for i,ticker in enumerate (tickers):
  print(i, ticker)
  try:
      data = yf.download(ticker, end=end_date, period='max',
                         interval = "1d",progress= True)
      data['Ticker'] = ticker
      data['Date'] = data.index.date

      for j in range(1, 31):
        data['growth_future_'+str(j)+'d'] = data['Adj Close'].shift(-j) / data['Adj Close']
      # ohlc_data[ticker] = data
      if stocks_df.empty:
        stocks_df = data[data['Date'] == data['Date'].min()]
      else:
        stocks_df = pd.concat([stocks_df, data[data['Date'] == data['Date'].min()]], ignore_index=True)

  except Exception as e:
      print(f"Error fetching data for ticker {ticker}: {e}")

df = stocks_df.loc[:, stocks_df.columns.str.startswith('growth')].describe()
df
# get max from growth_future.loc['75%']
p75 = df.loc['75%']
p75.idxmax(), p75.max()

0 IROH


[*********************100%%**********************]  1 of 1 completed


1 LGCB


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

2 ZKH
3 BAYA


...


[*********************100%%**********************]  1 of 1 completed

182 SDHC


183 ROMA


[*********************100%%**********************]  1 of 1 completed


('growth_future_28d', 1.0272727446122603)

# Question 3: Is Growth Concentrated in the Largest Stocks?
Get the share of days (percentage as int) when Large Stocks outperform (growth_7d - growth over 7 periods back) the Largest stocks?

Reuse [Code Snippet 5] to obtain OHLCV stats for 33 stocks for 10 full years of data (2014-01-01 to 2023-12-31). You'll need to download slightly more data (7 periods before 2014-01-01 to calculate the growth_7d for the first 6 days correctly):

US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

Now let's add the top 12-22 stocks (as of end-April 2024):

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

You should be able to obtain stats for 33 LARGEST STOCKS and 32 LARGE STOCKS (from the actual stats on Yahoo Finance)

Calculate growth_7d for every stock and every day. Get the average daily growth_7d for the LARGEST_STOCKS group vs. the LARGE_STOCKS group.

For example, for the first of data you should have:

Date	ticker_category	growth_7d
2014-01-01	LARGE	1.011684
2014-01-01	LARGEST	1.011797
On that day, the LARGEST group was growing faster than LARGE one (new stocks).

Calculate the number of days when the LARGE GROUP (new smaller stocks) outperforms the LARGEST GROUP, divide it by the total number of trading days (which should be 2595 days), and convert it to a percentage (closest INTEGER value). For example, if you find that 1700 out of 2595 days meet this condition, it means that 1700/2595 = 0.655, or approximately 66% of days, the LARGE stocks were growing faster than the LARGEST ones. This suggests that you should consider extending your dataset with more stocks to seek higher growth.

HINT: you can use pandas.pivot_table() to "flatten" the table (LARGE and LARGEST growth_7d as columns)

In [40]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']

EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']

INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']

LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS

NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']

NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']

NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']

LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA

In [41]:
# Define start and end dates
start_date = "2013-12-25"  # 7 periods before 2014-01-01
end_date = "2023-12-31"

# Function to calculate growth_7d
def calculate_growth_7d(data):
    return data.shift(-7) / data

In [64]:
# Fetch OHLCV data for LARGEST_STOCKS and LARGE_STOCKS
largest_stocks_data = yf.download(LARGEST_STOCKS, start=start_date, end=end_date)["Adj Close"]
large_stocks_data = yf.download(LARGE_STOCKS, start=start_date, end=end_date)["Adj Close"]
# Convert date columns to consistent format
largest_stocks_data.index = pd.to_datetime(largest_stocks_data.index)
large_stocks_data.index = pd.to_datetime(large_stocks_data.index)

[*********************100%%**********************]  33 of 33 completed
[*********************100%%**********************]  33 of 33 completed


In [65]:
# Calculate growth_7d for each group
largest_stocks_growth_7d = largest_stocks_data.apply(calculate_growth_7d)
large_stocks_growth_7d = large_stocks_data.apply(calculate_growth_7d)

In [66]:
large_stocks_growth_7d

Ticker,ADANIENT.NS,AI.PA,AIR.PA,ALV.DE,BAJFINANCE.NS,BUD,CDI.PA,COST,DTE.DE,EL.PA,...,PRX.AS,SNY,SU.PA,SUNPHARMA.NS,TATAMOTORS.NS,TITAN.NS,TSLA,UNH,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2013-12-26,0.934211,NaN,NaN,NaN,1.021666,0.976014,NaN,0.981202,NaN,NaN,...,NaN,0.995214,NaN,1.032199,0.991087,0.990650,0.945338,0.991850,0.997704,0.987711
2013-12-27,0.887890,0.961236,0.994132,0.994648,1.019116,0.985288,0.975610,0.977474,0.972222,1.015209,...,NaN,0.983144,0.977729,1.031687,0.981716,0.985182,0.988354,1.024367,0.999745,0.995665
2013-12-30,0.901972,0.966911,0.993181,0.992712,0.999679,0.995919,0.959445,0.960259,0.983508,1.056552,...,NaN,0.965778,0.971136,1.055136,0.987860,0.972920,0.992390,1.011233,0.989825,1.004287
2013-12-31,0.935428,0.955447,0.980290,NaN,0.980459,0.985628,0.927193,0.995715,NaN,1.064829,...,NaN,0.947604,0.973344,1.069698,0.977550,0.975381,0.980722,1.010359,0.992375,0.985771
2014-01-01,0.896558,NaN,NaN,NaN,0.993791,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.064733,0.978797,0.954289,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-12-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:

# Flatten the table using pivot_table
largest_stocks_pivot = pd.pivot_table(largest_stocks_growth_7d.unstack().reset_index(),
                                      values=0, index='Date')
large_stocks_pivot = pd.pivot_table(large_stocks_growth_7d.unstack().reset_index(),
                                    values=0, index='Date')

In [68]:
# Calculate average daily growth_7d for each group
avg_daily_growth_7d_largest = largest_stocks_pivot.mean(axis=1)
avg_daily_growth_7d_large = large_stocks_pivot.mean(axis=1)

In [69]:
avg_daily_growth_7d_largest

Date
2013-12-26    0.990353
2013-12-27    0.987984
2013-12-30    0.989096
2013-12-31    0.978946
2014-01-01    0.967214
                ...   
2023-12-13    1.013788
2023-12-14    1.014900
2023-12-15    1.013800
2023-12-18    1.009875
2023-12-19    1.004045
Length: 2592, dtype: float64

In [70]:
# Determine the number of days when large stocks outperform largest stocks
outperform_days = (avg_daily_growth_7d_large > avg_daily_growth_7d_largest).sum()

In [71]:
# Calculate the percentage of days when large stocks outperform largest stocks
total_days = len(avg_daily_growth_7d_largest)
percentage_outperform = int((outperform_days / total_days) * 100)

print("Percentage of days when LARGE stocks outperform LARGEST stocks:", percentage_outperform)

Percentage of days when LARGE stocks outperform LARGEST stocks: 46


# Question 4: Trying Another Technical Indicators strategy
What's the total gross profit (in THOUSANDS of $) you'll get from trading on CCI (no fees assumption)?

First, run the entire Colab to obtain the full DataFrame of data (after [Code Snippet 9]), and truncate it to the last full 10 years of data (2014-01-01 to 2023-12-31). If you encounter any difficulties running the Colab - you can download it using this link.

Let's assume you've learned about the awesome CCI indicator (Commodity Channel Index), and decided to use only it for your operations.

You defined the "defensive" value of a high threshould of 200, and you trade only on Fridays (Date.dt.dayofweek()==4).

That is, every time you see that CCI is >200 for any stock (out of those 33), you'll invest $1000 (each record when CCI>200) at Adj.Close price and hold it for 1 week (5 trading days) in order to sell at the Adj. Close price.

What's the expected gross profit (no fees) that you get in THOUSANDS $ (closest integer value) over many operations in 10 years? One operation calculations: if you invested $1000 and received $1010 in 5 days - you add $10 to gross profit, if you received $980 - add -$20 to gross profit. You need to sum these results over all trades (460 times in 10 years).

Additional:

Add an approximate fees calculation over the 460 trades from this calculator https://www.degiro.ie/fees/calculator (Product:"Shares, USA and Canada;" Amount per transaction: "1000 EUR"; Transactions per year: "460")
are you still profitable on those trades?

In [75]:
!gdown https://drive.google.com/uc?id=1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb

Downloading...
From: https://drive.google.com/uc?id=1m3Qisfs2XfWk6Sw_Uk5kHLWqwQ0q8SKb
To: /content/stocks_df_combined_trunc_2014_2023.parquet.brotli
100% 47.3M/47.3M [00:01<00:00, 28.3MB/s]


In [76]:
# pandas read df from downloaded file
df = pd.read_parquet('stocks_df_combined_trunc_2014_2023.parquet.brotli')
df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80762 entries, 0 to 80761
Columns: 202 entries, Open to growth_btc_usd_365d
dtypes: datetime64[ns](3), float64(128), int32(64), int64(5), object(2)
memory usage: 104.7+ MB


In [77]:
df

,Open,High,Low,Close,Adj Close_x,Volume,Ticker,Year,Month,Weekday,...,growth_brent_oil_7d,growth_brent_oil_30d,growth_brent_oil_90d,growth_brent_oil_365d,growth_btc_usd_1d,growth_btc_usd_3d,growth_btc_usd_7d,growth_btc_usd_30d,growth_btc_usd_90d,growth_btc_usd_365d
0,37.349998,37.400002,37.099998,37.160000,31.233059,30632200.0,MSFT,2014,2014-01-01,3,...,0.964302,0.992998,0.970030,1.158676,NaN,NaN,NaN,NaN,NaN,NaN
1,37.200001,37.220001,36.599998,36.910000,31.022930,31134800.0,MSFT,2014,2014-01-01,4,...,0.958139,0.984707,0.961500,1.143209,NaN,NaN,NaN,NaN,NaN,NaN
2,36.849998,36.889999,36.110001,36.130001,30.367352,43603700.0,MSFT,2014,2014-01-01,0,...,0.953798,0.998223,0.968951,1.168236,NaN,NaN,NaN,NaN,NaN,NaN
3,36.330002,36.490002,36.209999,36.410000,30.602673,35802800.0,MSFT,2014,2014-01-01,1,...,0.958653,0.993430,0.977598,1.097648,NaN,NaN,NaN,NaN,NaN,NaN
4,36.000000,36.139999,35.580002,35.759998,30.056356,59971700.0,MSFT,2014,2014-01-01,2,...,0.955161,0.973383,0.974977,1.100781,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80757,3424.000000,3496.000000,3408.600098,3477.949951,3477.949951,1681707.0,LT.NS,2023,2023-12-01,4,...,1.064772,0.971018,0.939967,0.797881,1.002935,1.040865,1.049324,1.175398,1.655339,2.614201
80758,3477.949951,3508.350098,3477.949951,3490.050049,3490.050049,1072263.0,LT.NS,2023,2023-12-01,1,...,1.058217,0.982429,0.956014,0.801404,0.974945,0.972127,1.005911,1.134509,1.613511,2.513055
80759,3510.000000,3549.000000,3504.149902,3544.000000,3544.000000,1389266.0,LT.NS,2023,2023-12-01,2,...,1.040496,0.965806,0.943050,0.749506,1.021694,1.009920,0.995203,1.166121,1.607712,2.598696
80760,3545.000000,3559.949951,3500.500000,3518.050049,3518.050049,3371121.0,LT.NS,2023,2023-12-01,3,...,1.005645,0.965632,0.932881,0.730228,0.981240,0.977409,0.971705,1.126794,1.583988,2.575301


In [79]:
df.columns.values

array(['Open', 'High', 'Low', 'Close', 'Adj Close_x', 'Volume', 'Ticker',
       'Year', 'Month', 'Weekday', 'Date', 'growth_1d', 'growth_3d',
       'growth_7d', 'growth_30d', 'growth_90d', 'growth_365d',
       'growth_future_5d', 'SMA10', 'SMA20', 'growing_moving_average',
       'high_minus_low_relative', 'volatility',
       'is_positive_growth_5d_future', 'ticker_type', 'index_x', 'adx',
       'adxr', 'apo', 'aroon_1', 'aroon_2', 'aroonosc', 'bop', 'cci',
       'cmo', 'dx', 'macd', 'macdsignal', 'macdhist', 'macd_ext',
       'macdsignal_ext', 'macdhist_ext', 'macd_fix', 'macdsignal_fix',
       'macdhist_fix', 'mfi', 'minus_di', 'mom', 'plus_di', 'dm', 'ppo',
       'roc', 'rocp', 'rocr', 'rocr100', 'rsi', 'slowk', 'slowd', 'fastk',
       'fastd', 'fastk_rsi', 'fastd_rsi', 'trix', 'ultosc', 'willr',
       'index_y', 'ad', 'adosc', 'obv', 'atr', 'natr', 'ht_dcperiod',
       'ht_dcphase', 'ht_phasor_inphase', 'ht_phasor_quadrature',
       'ht_sine_sine', 'ht_sine_leadsine', 

In [78]:
df.Date.min(), df.Date.max()

(Timestamp('2014-01-01 00:00:00'), Timestamp('2023-12-29 00:00:00'))

In [85]:
# Step 1: Filter the DataFrame to include only records where CCI > 200
cci_trades = df[df['cci'] > 200]

# Step 2: Initialize gross profit
gross_profit = 0

# Step 3: Calculate profit/loss for each trade
for index, row in cci_trades.iterrows():
    buy_price = row['Adj Close_x']
    sell_price = df.loc[index + 5, 'Adj Close_x']
    profit_loss = (sell_price - buy_price)   # Assuming $1000 investment
    gross_profit += profit_loss

# Step 4: Convert gross profit to thousands of dollars
gross_profit_thousands = int(gross_profit / 1000)

print("Total gross profit (in THOUSANDS of $):", gross_profit_thousands)

Total gross profit (in THOUSANDS of $): 2
